In [9]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]=""

In [11]:
import pandas as pd
from tqdm import tqdm
import torch
import sys
import time
import datetime
from itertools import product
import os
import numpy as np
from sklearn import tree
from sklearn import linear_model
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, f1_score
import matplotlib.pyplot as plt
from functools import partial
import shelve
import tqdm
import pickle
from collections import Counter

In [12]:
from ogb.graphproppred import Evaluator
from tdc.benchmark_group import admet_group

In [13]:
%autoreload 2
from huggingmolecules import MatConfig, MatFeaturizer, MatModel
from huggingmolecules import GroverConfig, GroverFeaturizer, GroverModel
from huggingmolecules import RMatConfig, RMatFeaturizer, RMatModel
from models import StrippedMatModel, StrippedGroverModel, StrippedRMatModel

In [14]:
%autoreload 2
from utils import *
from datasets import *
from predict_procedure import *

In [7]:
# from mat import MolecularAttentionTransformer

# model = MolecularAttentionTransformer()
# model.predict(["CN1CC[C@]23C(=O)C[C@@H]4C(=CCO[C@H]5CC(=O)N(c6ccccc62)[C@@H]3[C@@H]54)C1"]) ## tooks 11m 4s (!)

In [8]:
# group = admet_group(path = 'data/')
# benchmark = group.get("CYP2D6_Substrate_CarbonMangels")

# dataset = benchmark["test"]

# print(dataset[dataset.Drug == "CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C"].to_markdown())

# Output:
# |     | smiles                             |   Y |
# |----:|:-----------------------------------|----:|
# |  58 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   1 |
# | 226 | CCCCN1CCCC[C@H]1C(=O)Nc1c(C)cccc1C |   0 |else 'regression'

In [5]:
import pathlib

calculated = set(map(str, pathlib.Path("embeddings/").glob("*/*.zip")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("embedding_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if f"embeddings/{dataset_name}/{model_name}.zip" not in calculated:
            print(dataset_name, model_name, file=f)


In [8]:
import pathlib
done = set(map(str, pathlib.Path("results/").glob("*.pkl")))

pairs = product(get_dataset_names(), get_embedder_names())

with open("predict_params.txt", "w") as f:
    for dataset_name, model_name in pairs:
        if (f"results/{dataset_name}_{model_name}_rigid.pkl" not in done) or (f"results/{dataset_name}_{model_name}_forest.pkl" not in done):
            print(dataset_name, model_name, file=f)

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifierCV, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_curve, auc
from scipy.stats import spearmanr

In [63]:
def predict_test(df, split_idx, model, **kwargs):
    scaler = StandardScaler()

    targets = list(filter(lambda col: col not in ['smiles', "features", 'embeddings', 'Drug_ID'], df.columns))

    train_val = pd.concat([df.loc[split_idx["train"]], df.loc[split_idx["valid"]]])
    train_val = train_val.fillna(train_val[targets].mode().iloc[0])

    X = np.vstack(train_val.embeddings.to_numpy())
    y = train_val[targets].to_numpy()

    clf = model(kwargs).fit(scaler.fit_transform(X), y)

    test = df.loc[split_idx["test"]]
    X = np.vstack(test.embeddings.to_numpy())

    return clf.predict(scaler.transform(X))


# predict_regression_lin = partial(predict_test, model=lambda kwargs: linear_model.LinearRegression())

predict_regression_forest = partial(
    predict_test,
    model=lambda kwargs: GridSearchCV(
        RandomForestClassifier(
            n_estimators=100,
            criterion="entropy",
            refit=True,
            n_jobs=-1,
            random_state=0,
        ),
        {
            "min_samples_split": np.logspace(1, 5, 4, base=2).astype(int)
        },
        **kwargs
    )
)

predict_regression_rigid = partial(
    predict_test,
    model=lambda kwargs: ClassifierChain(
        RidgeCV(
            alphas=np.logspace(-2, 2, 10),
            cv=10,
        )
    )
)

# predict_classification_lin = partial(predict_test, model=lambda: ClassifierChain(LogisticRegression()))

predict_classification_forest = partial(
    predict_test,
    model=lambda kwargs: ClassifierChain(GridSearchCV(
        RandomForestClassifier(
            n_estimators=100,
            criterion="entropy",
            refit=True,
            n_jobs=-1,
            random_state=0,
        ),
        {
            "min_samples_split": np.logspace(1, 5, 4, base=2).astype(int)
        },
        **kwargs
    )))

predict_classification_rigid = partial(
    predict_test,
    model=lambda kwargs: ClassifierChain(
        RidgeClassifierCV(
            alphas=np.logspace(-2, 2, 10),
            cv=10,
            penalty="l1",
        )
    ))

In [83]:
dataset_name, base_model = "Caco2_Wang", "ChemBERTa-5M-MTR"

_, split_idx = get_dataset(dataset_name)
df = pd.read_pickle(f"embeddings/{dataset_name}/{base_model}.zip")

preds = predict_regression_forest(df, split_idx, scoring=get_metric(dataset_name))
preds

evaluate(dataset_name, preds)


Found local copy...
Loading...


Done!
Found local copy...
Found local copy...


('mae', 0.372)

In [54]:
def read(name):
    with open(f"results/{name}", "rb") as f:
        return pickle.load(f)

def spearmancorr_score(x, y):
    rho, pval = spearmanr(x, y, axis=0)
    return rho * (-1)


def pr_auc_score(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)


def get_metric(dataset):
    results = pd.DataFrame.from_records([read(name) for name in os.listdir("results/")])
    tmp1 = results.groupby("dataset").first().reset_index()[["dataset", "metric"]]
    tmp2 = {
        "roc-auc": "roc_auc",
        "rocauc": "roc_auc",
        "pr-auc": pr_auc,
        "mae": "neg_mean_absolute_error",
        "rmse": "neg_root_mean_squared_log_error",
        "spearman": spearmancorr,
        "ap": "average_precision",
    }
    return tmp2[dict(zip(tmp1.dataset, tmp1.metric))[dataset]]

# get_metric("asd")

In [82]:
dataset_name

'Caco2_Wang'

In [84]:
procedure(dataset_name, base_model)

Found local copy...
Loading...
Done!
Found local copy...


Found local copy...
Found local copy...
Loading...
Done!
Found local copy...
Found local copy...


In [87]:
read(f"{dataset_name}_{base_model}_rigid.pkl")

{'dataset': 'Caco2_Wang',
 'base_model': 'ChemBERTa-5M-MTR',
 'head_model': 'rigid',
 'metric': 'mae',
 'result': 0.333}

In [89]:
group = admet_group(path='data/')
group.dataset_names

Found local copy...


['caco2_wang',
 'hia_hou',
 'pgp_broccatelli',
 'bioavailability_ma',
 'lipophilicity_astrazeneca',
 'solubility_aqsoldb',
 'bbb_martins',
 'ppbr_az',
 'vdss_lombardo',
 'cyp2d6_veith',
 'cyp3a4_veith',
 'cyp2c9_veith',
 'cyp2d6_substrate_carbonmangels',
 'cyp3a4_substrate_carbonmangels',
 'cyp2c9_substrate_carbonmangels',
 'half_life_obach',
 'clearance_microsome_az',
 'clearance_hepatocyte_az',
 'herg',
 'ames',
 'dili',
 'ld50_zhu']